### Full-Lung CT and Cardiac CT Flip-Detection Classifier

For each subject of interest, load the lung mask and apply the classifier. The output will be None, 1, 2, or 3. None indicates no flip predicted, 1 indicates left-right flip predicted, 2 indicates top-bottom flip detected and 3 indicates front-back flip detected. 
 

In [2]:
# Import packages
import nibabel as nib
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd 
from nilearn import plotting
import seaborn as sns 
import random 
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from tqdm.auto import tqdm
from utils import ParallelMe
import joblib

##### Load Utility Functions

In [5]:
def max_slice_indices(lung):
    slice_areas_z = np.sum(lung, axis=(0, 1))
    largest_area_z_index = np.argmax(slice_areas_z)
    
    slice_areas_x = np.sum(lung, axis=(1, 2))
    largest_area_x_index = np.argmax(slice_areas_x)
    
    slice_areas_y = np.sum(lung, axis=(0, 2))
    largest_area_y_index = np.argmax(slice_areas_y)
    
    return largest_area_x_index, largest_area_y_index, largest_area_z_index

def pix_sum(lung, xlc, ylc, zlc):
    pix = np.zeros((3,2,2))
    xx,yy,zz = lung.shape
    
    # axial plane - slicing on z axis 
    for i in range(2):
        for j in range(2):
            pix[0,i,1-j] = lung[i*xx//2:(i+1)*xx//2, j*yy//2:(j+1)*yy//2, zlc].mean()
            
    # coronal plane - slicing on y axis          
    for i in range(2):
        for j in range(2):
            pix[1,1-i,j] = lung[i*xx//2:(i+1)*xx//2, ylc, j*zz//2:(j+1)*zz//2].mean()
            
    # saggital plane - slicing on x axis         
    for i in range(2):
        for j in range(2):
            pix[2,i,j] = lung[xlc, i*yy//2:(i+1)*yy//2, j*zz//2:(j+1)*zz//2].mean()
            
    return pix

##### Load The Classifier

In [15]:
# Load classifier 
lung_classifier = joblib.load('lung_flip_classifier.pkl')

# Load classifier 
cardiac_classifier = joblib.load('cardiac_flip_classifier.pkl')

In [16]:
def lung_flip_classify(lung): 
    X = []
    xlc, ylc, zlc = max_slice_indices(lung)
    pix = pix_sum(lung, xlc, ylc, zlc)
    X.append(pix.flatten())
    prediction = lung_classifier.predict(X)
    if prediction == 0:
        pass
    else: 
        print(f"{prediction} axis flip predicted for full-lung lung mask")
        return prediction

In [17]:
def cardiac_flip_classify(lung): 
    X = []
    xlc, ylc, zlc = max_slice_indices(lung)
    pix = pix_sum(lung, xlc, ylc, zlc)
    X.append(pix.flatten())
    prediction = cardiac_classifier.predict(X)
    if prediction == 0:
        pass
    else: 
        print(f"{prediction} axis flip predicted for cardiac lung mask")
        return prediction

##### Example Usage

In [21]:
# Define a directory in which the scans are stored 
SubDir = "/DATA/summer_students/process_AZ/cardiac_cleanup"

# Initiate list that stores prediction results 
predictions = list()

for sub in []:
    lung = nib.load(os.path.join(SubDir, f"{sub}_LUNG.nii.gz")).get_fdata()
    prediction = cardiac_flip_classify(lung)
    predictions.append(prediction)
    
print(predictions)

[None, None]
